In [1]:
# %% [markdown]
# # Automated Kinetic Data Processing Pipeline
# 
# **Description:** This notebook automates the processing of time-resolved fluorescence spectroscopy data. 
# It performs data cleaning, Savitzky-Golay smoothing, feature extraction (peak intensity, position, FWHM), 
# and compiles aggregate statistics for all reactions in the dataset.
#
# **Dependencies:** pandas, numpy, scipy, pathlib, tqdm, data_analysis (local module)

# %%
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm  # Provides progress bars

# Import local analysis module (refactored code)
import data_analysis as da

# %% [markdown]
# ### 1. Configuration and Parameters
# Define file paths and processing parameters here for reproducibility.

# %%
# Input Directory (Use 'r' for raw string to handle Windows backslashes)
# Update this path to the current experiment date
DATA_DIRECTORY = Path(r'\\Gfs01\g11\FluoSpec\Alle\Alex_Relling\Austausch\kinetic_setup_data\2025-12-01')

# Smoothing Parameters (Savitzky-Golay filter)
SMOOTH_WINDOW1 = 11
SMOOTH_POLY1 = 2

SMOOTH_WINDOW2 = 11
SMOOTH_POLY2 = 2

# Feature Extraction Thresholds
INTENSITY_THRESHOLD = 50.0  # Min counts to consider a real peak
TIME_THRESHOLD = 100.0      # Time (s) until which low intensity implies no peak

# Downsampling Settings (for plotting large datasets)
CUTOFF_TIME = 600           # Seconds (high res before this, low res after)
LATE_INTERVAL = 10          # Interval (s) for late-time data

# Snapshots to extract (in seconds)
SNAPSHOT_TIMES = []

# %% [markdown]
# ### 2. Load Metadata
# Identify reaction folders and load the corresponding experimental parameters.

# %%
# Identify valid reaction folders (assumes folder names start with a digit, e.g., '001_Reaction')
reaction_folders = [
    f.name for f in DATA_DIRECTORY.iterdir() 
    if f.is_dir() and f.name[0].isdigit()
]
print(f"Found {len(reaction_folders)} reaction folders.")

# Load Reaction Parameters (Frequency, Number of measurements, etc.)
try:
    # distinct file search to ensure we grab the right csv
    param_file = list(DATA_DIRECTORY.glob('reaction_parameters*.csv'))[0]
    reaction_params_df = pd.read_csv(param_file)
    print(f"Parameters loaded from: {param_file.name}")
except IndexError:
    raise FileNotFoundError("CRITICAL: 'reaction_parameters.csv' not found in directory.")

# %% [markdown]
# ### 3. Primary Processing Loop
# Iterate through each folder to clean, smooth, and extract features.
# *Note: Processing is grouped per folder to minimize network I/O latency.*

# %%
print("Starting individual reaction processing...")

for folder in tqdm(reaction_folders, desc="Processing Reactions"):
    
    # A. Standardize Time Axis (Clean)
    # Aligns raw data columns to theoretical time points based on frequency
    da.standardize_time_axis(
        directory=DATA_DIRECTORY, 
        folder=folder, 
        reaction_params_df=reaction_params_df
    )
    
    # B. Spectral Smoothing
    # Applies Savitzky-Golay filter along the wavelength axis
    da.apply_smoothing(
        directory=DATA_DIRECTORY, 
        folder=folder, 
        window_length=SMOOTH_WINDOW1, 
        polyorder=SMOOTH_POLY1
    )

    da.merge_vis_nir_spectra(
        directory=DATA_DIRECTORY,
        folder=folder,
        stitch_wavelength=930.0,    # Adjust if needed
        stitch_window=10.0,         # Adjust if needed
        min_signal_threshold=50.0    # Adjust if noise is higher
    )

    # Generates heatmaps immediately after smoothing
    da.plot_reaction_heatmap(
        directory=DATA_DIRECTORY,
        folder=folder
    )
    # C. Data Reduction (Downsampling)
    # Creates smaller files for quick visualization
    da.downsample_temporal_data(
        directory=DATA_DIRECTORY, 
        folder=folder, 
        cutoff_time=CUTOFF_TIME, 
        late_interval=LATE_INTERVAL
    )
    
    # D. Feature Extraction
    # Calculates Peak Max, Peak Position (nm), and FWHM (eV)
    da.extract_spectral_features(
        directory=DATA_DIRECTORY, 
        folder=folder,
        intensity_threshold=INTENSITY_THRESHOLD,
        time_threshold=TIME_THRESHOLD
    )

print("Individual processing complete.")

# %% [markdown]
# ### 4. Global Aggregation
# Compile the extracted features from all reactions into summary datasets for comparative analysis.

# %%
print("Compiling global experiment traces...")

da.compile_experiment_traces(
    directory=DATA_DIRECTORY, 
    folders=reaction_folders,
    window_length=SMOOTH_WINDOW2,
    polyorder=SMOOTH_POLY2
)

print(f"Summary files saved to: {DATA_DIRECTORY}")

# %% [markdown]
# ### 5. Snapshot Extraction
# Extract full spectral data at specific time points (defined in config) for all reactions. 
# Creates normalized and raw intensity files.

# %%
print(f"Extracting snapshots for times: {SNAPSHOT_TIMES} s")

da.extract_snapshots_at_times(
    directory=DATA_DIRECTORY, 
    folders=reaction_folders, 
    target_timestamps=SNAPSHOT_TIMES
)

print("Workflow finished successfully.")

Found 1 reaction folders.
Parameters loaded from: reaction_parameters_2025-12-01_15-36-13.csv
Starting individual reaction processing...


Processing Reactions:   0%|          | 0/1 [00:00<?, ?it/s]

Warning in 01_10-0_Dil-1_f1_2025-12-01_15-36-13/Emission_nir: Mismatched data length. Adjusting to fit theoretical time points.
  -> Cut off 501 wavelengths < 500nm for 01_10-0_Dil-1_f1_2025-12-01_15-36-13/Emission_vis
Warning in 01_10-0_Dil-1_f1_2025-12-01_15-36-13/Emission_vis: Mismatched data length. Adjusting to fit theoretical time points.


c:\Users\arelling\Desktop\git\kinetic-setup\publication\data_analysis\data_analysis.py:201: RuntimeWarning: divide by zero encountered in divide
  energies = HC_CONST / wavelengths


Individual processing complete.
Compiling global experiment traces...
Compiling global datasets...
Summary files saved to: \\Gfs01\g11\FluoSpec\Alle\Alex_Relling\Austausch\kinetic_setup_data\2025-12-01
Extracting snapshots for times: [] s
Workflow finished successfully.
